In [116]:
### SETUP CLASS to SUBMIT M2M REQUESTS

import requests

class MachineToMachine(object):
    def __init__(self, base_url, api_user, api_key, username='danmergens@gmail.com'):
        self.base_url = base_url
        self.api_user = api_user
        self.api_key = api_key
        self.username = username
        self.auth = (api_user, api_key)
        self.inv_url = self.base_url + '/api/m2m/12576/sensor/inv'
        self.ingest_url = self.base_url + '/api/m2m/12589/ingestrequest'

    def get(self, url, params=None):  # TODO remove
        response = requests.get(url, auth=self.auth, params=params)
        if response.status_code != requests.codes.ok:
            print('request failed (%s) for %s - %s' % (response.reason, url, params))
            return None
        return response.json()

    def toc(self):
        """table of contents"""
        url = '/'.join((self.inv_url, 'toc'))
        return requests.get(url).json()

    def node_inventory(self, subsite, node):
        """returns sensors on the specified node"""
        url = '/'.join((self.inv_url, subsite, node))
        return ['-'.join((subsite, node, sensor)) for sensor in self.get(url)]

    def streams(self):
        """returns list of all streams"""
        toc = self.toc()
        stream_map = {}
        toc = toc['instruments']
        for row in toc:
            rd = row['reference_designator']
            for each in row['streams']:
                stream_map.setdefault(rd, {}).setdefault(each['method'], set()).add(each['stream'])
        return stream_map
    
    def instruments(self):
        nodes = []
        for subsite in self.get(self.inv_url):
            for node in self.get('/'.join((self.inv_url, subsite))):
                nodes.extend(self.node_inventory(subsite, node))
        return nodes
    
    def inv_request(self, refdes, method=None, stream=None, payload=None):
        subsite, node, inst = refdes.split('-', 2)
        url = '/'.join((self.inv_url, subsite, node, inst))
        if method:
            url = '/'.join((url, method))
            if stream:
                url = '/'.join((url, stream))
        return self.get(url, params=payload)
    
    def playback(self, payload):
        """create a request for playback ingest"""
        return requests.post(self.ingest_url, auth=(self.api_user, self.api_key), json=payload)
    
    def ingest_status(self, request_id):
        """get the current status of an ingest request"""
        response = requests.get('/'.join((self.ingest_url, str(request_id))), auth=self.auth)
        return response
        # if response.status_code == 200:
        #     return response.json()['status']
        # else:
        #     print('invalid ingest request: %d' % request_id)
        #     return None
        
    def ingest_jobs(self, request_id):
        """get the status of all jobs created by an ingest request"""
        payload = { 'ingestRequestId': request_id }
        response = requests.get('/'.join((self.ingest_url, 'jobs')), auth=self.auth, params=payload)
        return response

    def ingest_job_counts(self, request_id):
        """get overall status on files processed by an ingest request"""
        payload = { 'ingestRequestId': request_id, 'groupBy': 'status' }
        response = requests.get('/'.join((self.ingest_url, 'jobcounts')), auth=self.auth, params=payload)
        return response

    def purge(self, refdes):
        """purge all data for a particular reference designator""" # TODO test
        subsite, node, sensor = refdes.split('-', 2)
        payload = {
            'username': self.username,
            'subsite': subsite,
            'node': node,
            'sensor': sensor
        }
        response = requests.put('/'.join((self.ingest_url, 'purgerecords')), auth=self.auth, json=payload)
        return response

    def metadata_times(self, refdes):
        """fetch the stream metadata for an instrument"""
        subsite, node, inst = refdes.split('-', 2)
        return self.get('/'.join((self.inv_url, subsite, node, inst, 'metadata/times')))
    
    def availability(self, refdes):
        return self.get('/'.join((self.base_url, 'api/data_availability', refdes)))
    
    def context(self):
        return '%s %s' % (self.base_url, self.username)

In [117]:
### SETUP CONTEXT for M2M USER

config_prod = {
    'url': 'https://ooinet.oceanobservatories.org',
    'apiname': 'OOIAPI-2WGGC3R53I8YGZ',
    'apikey': 'EOW97ASHSJVKWL'}
config_test = {
    'url': 'https://ooinet-dev-01.oceanobservatories.org',
     'apiname': 'OOIAPI-N7MHVW7E72PI18',
     'apikey': 'IDDQZY0RLNXF6C'}
config_uft3 = {
    'url': 'https://ooinet-dev-03.oceanobservatories.org',
     'apiname': 'OOIAPI-XI28GSIO9LW25G',
     'apikey': 'VXW293BA6CQUNS'}

config = config_uft3
m2m = MachineToMachine(config['url'], config['apiname'], config['apikey'])
m2m.context()
user = 'danmergens@gmail.com'

In [118]:
### MAP the CABLED PLAYBACK FORMATS and DRIVERS

# only need to enumerate playback formats that are not DATALOG (default) 
# ZPLSC - should be used for all ZPLSC files
# DATALOG - "should" be format for all non-ZPLSC files since 2016
# CHUNKY - pre-2016 format for files with binary data
# ASCII - pre-2016 for files containing ASCII characters

playback_formats = {
    'BOTPT': 'CHUNKY',
    'D1000': 'CHUNKY',
    'VEL3D': 'CHUNKY',
    'ZPLSC': 'ZPLSC',
}

def playback_format(refdes):
    sensor = refdes.split('-')[3]
    sensor_type = sensor[0:5]
    file_format = playback_formats.get(sensor_type, None)
    # default format is DATALOG, however earlier raw datafiles can vary based on time (TODO)
    if not file_format:
        file_format = 'DATALOG'
    return file_format

for refdes in ['CE02SHBP-LJ01D-05-ADCPTB104', 'RS03ASHS-MJ03B-09-BOTPTA304']:
    print(refdes, playback_format(refdes))

parser_drivers = {
    'ADCPS': 'mi.instrument.teledyne.workhorse.adcp.driver',
    'ADCPT': 'mi.instrument.teledyne.workhorse.adcp.driver',
    'BOTPT': 'mi.instrument.noaa.botpt.ooicore.driver',
    'CTDBP': 'mi.instrument.seabird.sbe16plus_v2.ctdbp_no.driver',
    'CTDPFA': 'mi.instrument.seabird.sbe16plus_v2.ctdpf_sbe43.driver',
    'CTDPFB': 'mi.instrument.seabird.sbe16plus_v2.ctdpf_jb.driver',
    'D1000': 'mi.instrument.mclane.ras.d1000.driver',
    'FLORD': 'mi.instrument.wetlabs.fluorometer.flort_d.driver',
    'FLORT': 'mi.instrument.wetlabs.fluorometer.flort_d.driver',
    'HPIES': 'mi.instrument.uw.hpies.ooicore.driver',
    'NUTNR': 'mi.instrument.satlantic.suna_deep.ooicore.driver',
    'OPTAA': 'mi.instrument.wetlabs.ac_s.ooicore.driver',
    'PARAD': 'mi.instrument.satlantic.par_ser_600m.driver',
    'PCO2WA': 'mi.instrument.sunburst.sami2_pco2.pco2a.driver',
    'PCO2WB': 'mi.instrument.sunburst.sami2_pco2.pco2b.driver',
    'PHSEN': 'mi.instrument.sunburst.sami2_ph.ooicore.driver',
    'PREST': 'mi.instrument.seabird.sbe54tps.driver',
    'SPKIR': 'mi.instrument.satlantic.ocr_507_icsw.ooicore.driver',
    'TMPSF': 'mi.instrument.rbr.xr_420_thermistor_24.ooicore.driver',
    'TRHPH': 'mi.instrument.uw.bars.ooicore.driver',
    'VEL3DB': 'mi.instrument.nobska.mavs4.ooicore.driver',
    'VEL3DC': 'mi.instrument.nortek.vector.ooicore.driver',
    'VELPT': 'mi.instrument.nortek.aquadopp.ooicore.driver',
    'CE04OSPS-PC01B-4A-CTDPFA109': 'mi.instrument.seabird.sbe16plus_v2.ctdpf_jb.driver',
    'RS01SBPS-PC01A-4A-CTDPFA103': 'mi.instrument.seabird.sbe16plus_v2.ctdpf_jb.driver',
    'RS01SBPS-PC01A-06-VADCPA101MAIN': 'mi.instrument.teledyne.workhorse.vadcp.playback4',
    'RS01SBPS-PC01A-06-VADCPA101-5TH': 'mi.instrument.teledyne.workhorse.vadcp.playback5',
}

def parser_driver(refdes):
    # check for exceptional case first (uses entire refdes)
    driver = parser_drivers.get(refdes, None)
    if not driver:
        sensor = refdes.split('-')[3]
        sensor_type = sensor[0:5]
        driver = parser_drivers.get(sensor_type, None)
        if driver is None:
            sensor_type = sensor[0:6]
            driver = parser_drivers.get(sensor_type, None)
    return driver

print(parser_driver('RS01SBPS-SF01A-4F-PCO2WA101'))
print(parser_driver('CE02SHBP-LJ01D-09-PCO2WB103'))
print(parser_driver('RS03INT2-MJ03D-12-VEL3DB304'))
print(parser_driver('CE04OSPS-PC01B-4A-CTDPFA109'))  # 'ctdpf_optode_sample'
print(parser_driver('RS01SBPS-PC01A-4A-CTDPFA103'))  # 'ctdpf_optode_sample'
print(parser_driver('RS01SBPS-PC01A-06-VADCPA101MAIN'))
print(parser_driver('RS01SBPS-PC01A-06-VADCPA101-5TH'))

CE02SHBP-LJ01D-05-ADCPTB104 DATALOG
RS03ASHS-MJ03B-09-BOTPTA304 CHUNKY
mi.instrument.sunburst.sami2_pco2.pco2a.driver
mi.instrument.sunburst.sami2_pco2.pco2b.driver
mi.instrument.nobska.mavs4.ooicore.driver
mi.instrument.seabird.sbe16plus_v2.ctdpf_jb.driver
mi.instrument.seabird.sbe16plus_v2.ctdpf_jb.driver
mi.instrument.teledyne.workhorse.vadcp.playback4
mi.instrument.teledyne.workhorse.vadcp.playback5


In [119]:
### CREATE the CABLED INGEST REQUEST BODY

def cabledRequestFactory(username, refdes, filemasks, file_range=None, data_range=None, force=False, priority=5, 
                         max_files=None):
    subsite, node, sensor = refdes.split('-', 2)
    request = {
        'username': username,
        'state': 'RUN',
        'type': 'PLAYBACK',  # 'RECOVERED', 'TELEMETERED', 'PLAYBACK'
        'options': {
        },
        'ingestRequestFileMasks': [],
    }
    request['priority'] = priority
    if max_files:
        request['maxNumFiles'] = max_files
    request['options']['format'] = playback_format(refdes)

    driver = parser_driver(refdes)
    if driver is None:
        print('unable to find driver for sensor: ', refdes)
        return None

    for mask in filemasks:
        request['ingestRequestFileMasks'].append(
        {
            'dataSource': 'streamed',
            'refDes': {
                'full': True,
                'node': node,
                'sensor': sensor,
                'subsite': subsite,
            },
            'refDesFinal': True,
            'parserDriver': driver,
            'fileMask': mask,
            'deployment': 0,  # always 0 for cabled playback
        })

    if file_range:
        request['options']['beginFileDate'] = file_range[0]
        request['options']['endFileDate'] = file_range[1]

    if data_range:
        request['options']['beginData'] = data_range[0]
        request['options']['endData'] = data_range[1]
    
    if force:
        request['options']['checkExistingFiles'] = False
    
    return request

In [120]:
# FILEMASK FORMAT for CABLED ARCHIVE FILES
cabled_filemask_format = '/rsn_cabled/rsn_data/DVT_Data/{node}/{sensor}/{year}/{month:02d}/{sensor}_*_{year}{month:02d}{day:02d}*_UTC.dat'

# filemask_glob_format9 = '/rsn_cabled/rsn_data/DVT_Data/{node}/{sensor}/2018/09/{sensor}*_201809{{1[3-9],[23]?}}*_UTC.dat'  # doesn't work
# filemask_glob_format10 = '/rsn_cabled/rsn_data/DVT_Data/{node}/{sensor}/2018/10/{sensor}*_201810{{0?,10}}*_UTC.dat'  # doesn't work
# filemask_glob_format = '/rsn_cabled/rsn_data/DVT_Data/{node}/{sensor}/2018/{{09,10}}/{sensor}*_UTC.dat'  # doesn't work
filemask_glob_format = '/rsn_cabled/rsn_data/DVT_Data/{node}/{sensor}/2019/04/{sensor}*_UTC.dat'
filemask_glob_format_pre = '/rsn_cabled/rsn_data/DVT_Data/{node}/{sensor}/2018/09/{sensor}*20180912*_UTC.dat'
filemask_glob_format_post = '/rsn_cabled/rsn_data/DVT_Data/{node}/{sensor}/2018/10/{sensor}*20181011*_UTC.dat'
remapped_sensors = {
    'D1000A301': 'RASFLA301_D1000',
}

# prior to 2017-08-10, files are stored in the top level directory
filemask_glob_old = '/rsn_cabled/rsn_data/DVT_Data/{node}/{sensor}*.dat'
filemask_glob_new = '/rsn_cabled/rsn_data/DVT_Data/{node}/{sensor}/*/*/{sensor}*_UTC.dat'

def create_filemasks(refdes):
    """file globs specific to the sub-range from the most recent outage"""
    _, node, _, sensor = refdes.split('-', 3)
    filemasks = []
    if sensor in remapped_sensors.keys():
        sensor = remapped_sensors[sensor]
    if type(sensor) != list:
        sensor = [sensor]
    # print(sensor)
    for s in sensor:
        #filemasks.append(filemask_glob_format.format(node=node.lower(), sensor=s))
        #filemasks.append(filemask_glob_old.format(node=node.lower(), sensor=s))
        filemasks.append(filemask_glob_new.format(node=node.lower(), sensor=s))
    return filemasks

print(create_filemasks('RS03INT1-MJ03C-07-D1000A301'))
print(create_filemasks('RS01SBPS-PC01A-06-VADCPA101MAIN'))
print(create_filemasks('RS01SBPS-PC01A-06-VADCPA101-5TH'))

def run_playback(user, refdes, file_range=None, data_range=None, force=False):
    filemasks = create_filemasks(refdes)
    request = cabledRequestFactory(user, refdes, filemasks, file_range=file_range, 
                                   data_range=data_range, force=force)
    # print(json.dumps(request, indent=4))
    response = m2m.playback(request)
    return response

['/rsn_cabled/rsn_data/DVT_Data/mj03c/RASFLA301_D1000/*/*/RASFLA301_D1000*_UTC.dat']
['/rsn_cabled/rsn_data/DVT_Data/pc01a/VADCPA101MAIN/*/*/VADCPA101MAIN*_UTC.dat']
['/rsn_cabled/rsn_data/DVT_Data/pc01a/VADCPA101-5TH/*/*/VADCPA101-5TH*_UTC.dat']


In [122]:
### LIST of all CABLED INSTRUMENTS (current in OOINet, except for hydrophones and seismometers)
cabled_refdes = [
    'CE02SHBP-LJ01D-05-ADCPTB104', 
    'CE02SHBP-LJ01D-06-CTDBPN106', 
    'CE02SHBP-LJ01D-06-DOSTAD106', 
    'CE02SHBP-LJ01D-07-VEL3DC108',  # failed to get recent data
    'CE02SHBP-LJ01D-08-OPTAAD106', 
    'CE02SHBP-LJ01D-09-PCO2WB103', 
    'CE02SHBP-LJ01D-10-PHSEND103', 
    'CE04OSBP-LJ01C-05-ADCPSI103', 
    'CE04OSBP-LJ01C-06-CTDBPO108',
    'CE04OSBP-LJ01C-06-DOSTAD108',
#    'CE04OSBP-LJ01C-07-VEL3DC107',  # failed to get recent data
    'CE04OSBP-LJ01C-08-OPTAAC104',
    'CE04OSBP-LJ01C-09-PCO2WB104',
    'CE04OSBP-LJ01C-10-PHSEND107',
    'CE04OSPS-PC01B-4A-CTDPFA109',
    'CE04OSPS-PC01B-4A-DOSTAD109',
    'CE04OSPS-PC01B-4B-PHSENA106', 
    'CE04OSPS-PC01B-4D-PCO2WA105',
#    'CE04OSPS-PC01B-4C-ZPLSCB102',  # offline since 2018-10
    'CE04OSPS-SF01B-2A-CTDPFA107', 
    'CE04OSPS-SF01B-2B-PHSENA108',
    'CE04OSPS-SF01B-3A-FLORTD104', 
    'CE04OSPS-SF01B-3B-OPTAAD105', 
    'CE04OSPS-SF01B-3C-PARADA102', 
    'CE04OSPS-SF01B-3D-SPKIRA102', 
    'CE04OSPS-SF01B-4A-NUTNRA102', 
    'CE04OSPS-SF01B-4F-PCO2WA102', 
#    'CE04OSPS-SF01B-4B-VELPTD106',  # failed to get recent data
    'RS01SBPS-PC01A-05-ADCPTD102', 
    'RS01SBPS-PC01A-06-VADCPA101', 
    'RS01SBPS-PC01A-4A-CTDPFA103', 
    'RS01SBPS-PC01A-4A-DOSTAD103', 
    'RS01SBPS-PC01A-4B-PHSENA102', 
    'RS01SBPS-PC01A-4C-FLORDD103',
    'RS01SBPS-SF01A-2A-CTDPFA102',
    'RS01SBPS-SF01A-2D-PHSENA101',
    'RS01SBPS-SF01A-3A-FLORTD101',
    'RS01SBPS-SF01A-3B-OPTAAD101', 
    'RS01SBPS-SF01A-3C-PARADA101', 
    'RS01SBPS-SF01A-3D-SPKIRA101',
    'RS01SBPS-SF01A-4A-NUTNRA101',
    'RS01SBPS-SF01A-4B-VELPTD102',
    'RS01SBPS-SF01A-4F-PCO2WA101',
    'RS01SLBS-LJ01A-05-HPIESA101', 
    'RS01SLBS-LJ01A-10-ADCPTE101',
    'RS01SLBS-LJ01A-11-OPTAAC103', 
    'RS01SLBS-LJ01A-12-CTDPFB101',
    'RS01SLBS-LJ01A-12-DOSTAD101',
    'RS01SLBS-MJ01A-06-PRESTA101', 
    'RS01SLBS-MJ01A-12-VEL3DB101',
    'RS01SUM1-LJ01B-09-PRESTB102', 
    'RS01SUM1-LJ01B-12-VEL3DB104', 
    'RS01SUM2-MJ01B-12-ADCPSK101',
    'RS03ASHS-MJ03B-07-TMPSFA301', 
    'RS03ASHS-MJ03B-09-BOTPTA304', 
    'RS03ASHS-MJ03B-10-CTDPFB304',  # should there be a matching DOSTA?
    'RS03AXBS-LJ03A-05-HPIESA301', 
    'RS03AXBS-LJ03A-10-ADCPTE303', 
    'RS03AXBS-LJ03A-11-OPTAAC303', 
    'RS03AXBS-LJ03A-12-CTDPFB301',
    'RS03AXBS-LJ03A-12-DOSTAD301',
    'RS03AXBS-MJ03A-06-PRESTA301', 
    'RS03AXBS-MJ03A-12-VEL3DB301',
    'RS03CCAL-MJ03F-05-BOTPTA301', 
    'RS03ECAL-MJ03E-06-BOTPTA302',
    'RS03INT1-MJ03C-07-D1000A301', 
#    'RS03INT1-MJ03C-10-TRHPHA301',  # failed to get recent data
#    'RS03INT1-MJ03C-09-THSPHA301',  # no data since 2015-07-11
    'RS03INT2-MJ03D-06-BOTPTA303', 
    'RS03INT2-MJ03D-12-VEL3DB304',
]

In [123]:
### DETERMINE DATA GAP
avail = m2m.availability('CE02SHBP-LJ01D-06-CTDBPN106')
dp_avail = [x['data'] for x in avail['availability'] if 'Data Products' in x['measure']][0]
[x for x in dp_avail if 'Missing' in x and '2019-07-26' in x[0]]

[]

In [124]:
### MAP SCIENCE STREAM NAME TO INSTRUMENT
science_streams = {
    'ADCPS': 'adcp_velocity_beam',
    'ADCPT': 'adcp_velocity_beam',
    'BOTPT': 'botpt_nano_sample',
    'CTDBP': 'ctdbp_no_sample',
    'CTDPFA': 'ctdpf_sbe43_sample',
    'DOSTA': 'do_stable_sample',
    'CE04OSPS-PC01B-4A-CTDPFA109': 'ctdpf_optode_sample',
    'RS01SBPS-PC01A-4A-CTDPFA103': 'ctdpf_optode_sample',
    'CTDPFB': 'ctdpf_optode_sample',
    'D1000': 'd1000_sample',
    'FLORD': 'flort_d_data_record',
    'FLORT': 'flort_d_data_record',
    'HPIES': 'horizontal_electric_field',
    'NUTNR': 'nutnr_a_sample',
    'OPTAA': 'optaa_sample',
    'PARAD': 'parad_sa_sample',
    'PCO2WA': 'pco2w_a_sami_data_record',
    'PCO2WB': 'pco2w_b_sami_data_record',
    'PHSEN': 'phsen_data_record',
    'PREST': 'prest_real_time',
    'SPKIRA': 'spkir_data_record',
    'TMPSF': 'tmpsf_sample',
    'TRHPH': 'trhph_sample',
    'VADCP': 'vadcp_velocity_beam',
    'VEL3D': 'vel3d_b_sample',
    'VELPT': 'velpt_velocity_data',
}

def science_stream(refdes):
    stream = science_streams.get(refdes, None)
    sensor_type = refdes.split('-')[3][0:5]
    if not stream: # try 5 sensor_type code
        stream = science_streams.get(sensor_type, None)
        if not stream: # second pass for 6 character specific streams (e.g. PCO2WA)
            sensor_type = refdes.split('-')[3][0:6]
            stream = science_streams.get(sensor_type, None)
    return stream

for refdes in ['RS01SBPS-PC01A-4A-CTDPFA103', 'RS01SBPS-SF01A-2A-CTDPFA102', 'RS01SBPS-SF01A-2D-PHSENA101']:
    print(refdes, science_stream(refdes))

def check_inventory(refdes, data_range):
    payload = {
        'user': 'dmergens',
        'limit': 30
    }
    if data_range:
        payload['beginDT'] = data_range[0]
        payload['endDT'] = data_range[1]

    sensor_type = refdes.split('-')[3][0:5]
    method = 'streamed'
    stream = science_stream(refdes)
    response = m2m.inv_request(refdes, method, stream, payload)
    if not stream:
        print('add appropriate stream from the following list to science_streams: %s for %s' % (response, sensor_type))
    return response

RS01SBPS-PC01A-4A-CTDPFA103 ctdpf_optode_sample
RS01SBPS-SF01A-2A-CTDPFA102 ctdpf_sbe43_sample
RS01SBPS-SF01A-2D-PHSENA101 phsen_data_record


In [125]:
### CHECK DATA AVAILABILITY
current = ('2019-08-20T00:00:00.000Z', '2019-08-21T00:00:00.000Z')
july_gap = ('2019-07-26T11:17:12.000Z', '2019-07-29T16:04:42.000Z')
for refdes in cabled_refdes:
    response = check_inventory(refdes, july_gap)
    samples = len(response) if response else 'missing'
    print(refdes, samples)

request failed (NOT FOUND) for https://ooinet-dev-03.oceanobservatories.org/api/m2m/12576/sensor/inv/CE02SHBP/LJ01D/05-ADCPTB104/streamed/adcp_velocity_beam - {'user': 'dmergens', 'limit': 30, 'beginDT': '2019-07-26T11:17:12.000Z', 'endDT': '2019-07-29T16:04:42.000Z'}
CE02SHBP-LJ01D-05-ADCPTB104 missing
CE02SHBP-LJ01D-06-CTDBPN106 30
request failed (BAD REQUEST) for https://ooinet-dev-03.oceanobservatories.org/api/m2m/12576/sensor/inv/CE02SHBP/LJ01D/06-DOSTAD106/streamed/do_stable_sample - {'user': 'dmergens', 'limit': 30, 'beginDT': '2019-07-26T11:17:12.000Z', 'endDT': '2019-07-29T16:04:42.000Z'}
CE02SHBP-LJ01D-06-DOSTAD106 missing
request failed (BAD REQUEST) for https://ooinet-dev-03.oceanobservatories.org/api/m2m/12576/sensor/inv/CE02SHBP/LJ01D/07-VEL3DC108/streamed/vel3d_b_sample - {'user': 'dmergens', 'limit': 30, 'beginDT': '2019-07-26T11:17:12.000Z', 'endDT': '2019-07-29T16:04:42.000Z'}
CE02SHBP-LJ01D-07-VEL3DC108 missing
CE02SHBP-LJ01D-08-OPTAAD106 27
CE02SHBP-LJ01D-09-PCO2WB1

In [114]:
# PLAYBACK
july_gap_dates = ('2019-07-26', '2019-07-30')
for refdes in cabled_refdes:
    response = run_playback(user, refdes, file_range=july_gap_dates, data_range=july_gap, force=False)
    if response:
        print(refdes, response.json())
    else:
        print('no response for playback request of %s' % refdes)

CE02SHBP-LJ01D-05-ADCPTB104 {'message': 'Element created successfully.', 'id': 385, 'statusCode': 'CREATED'}
CE02SHBP-LJ01D-06-CTDBPN106 {'message': 'Element created successfully.', 'id': 386, 'statusCode': 'CREATED'}
unable to find driver for sensor:  CE02SHBP-LJ01D-06-DOSTAD106
no response for playback request of CE02SHBP-LJ01D-06-DOSTAD106
CE02SHBP-LJ01D-08-OPTAAD106 {'message': 'Element created successfully.', 'id': 387, 'statusCode': 'CREATED'}
CE02SHBP-LJ01D-09-PCO2WB103 {'message': 'Element created successfully.', 'id': 388, 'statusCode': 'CREATED'}
CE02SHBP-LJ01D-10-PHSEND103 {'message': 'Element created successfully.', 'id': 389, 'statusCode': 'CREATED'}
CE04OSBP-LJ01C-05-ADCPSI103 {'message': 'Element created successfully.', 'id': 390, 'statusCode': 'CREATED'}
CE04OSBP-LJ01C-06-CTDBPO108 {'message': 'Element created successfully.', 'id': 391, 'statusCode': 'CREATED'}
unable to find driver for sensor:  CE04OSBP-LJ01C-06-DOSTAD108
no response for playback request of CE04OSBP-LJ

In [129]:
# job status
for job in range(385, 441):
# for job in [385]:
    #print(job, [x['status'] for x in m2m.ingest_jobs(job).json()])   # debug
    #print(job, json.dumps(m2m.ingest_jobs(job).json(), indent=4))    # debug
    #print(job, json.dumps(m2m.ingest_status(job).json(), indent=4))  # debug
    counts = {}
    job_status = m2m.ingest_jobs(job).json()
    #with open('job_status.json', 'w') as file:
        #file.write(json.dumps(job_status))
    for x in job_status:
        status = x['status']
        prev_count = counts.get(status, None)
        if prev_count is None:
            counts[status] = 1
        else:
            counts[status] = prev_count + 1
        if status in [u'ERROR', u'WARNING']:
            print(x['status'], x['filePath'])
    print(job, counts)

ERROR /rsn_cabled/rsn_data/DVT_Data/lj01d/ADCPTB104/2019/07/ADCPTB104_10.33.14.5_2101_20190727T0000_UTC.dat
ERROR /rsn_cabled/rsn_data/DVT_Data/lj01d/ADCPTB104/2019/07/ADCPTB104_10.33.14.5_2101_20190728T0000_UTC.dat
ERROR /rsn_cabled/rsn_data/DVT_Data/lj01d/ADCPTB104/2019/07/ADCPTB104_10.33.14.5_2101_20190726T0000_UTC.dat
385 {'ERROR': 3}
386 {'COMPLETE': 3}
387 {'COMPLETE': 3}
388 {'COMPLETE': 3}
389 {'COMPLETE': 3}
ERROR /rsn_cabled/rsn_data/DVT_Data/lj01c/ADCPSI103/2019/07/ADCPSI103_10.33.12.5_2101_20190728T0000_UTC.dat
ERROR /rsn_cabled/rsn_data/DVT_Data/lj01c/ADCPSI103/2019/07/ADCPSI103_10.33.12.5_2101_20190727T0000_UTC.dat
ERROR /rsn_cabled/rsn_data/DVT_Data/lj01c/ADCPSI103/2019/07/ADCPSI103_10.33.12.5_2101_20190726T0000_UTC.dat
390 {'ERROR': 3}
391 {'COMPLETE': 3}
392 {'COMPLETE': 3}
393 {'COMPLETE': 3}
394 {'COMPLETE': 3}
395 {'COMPLETE': 3}
396 {'COMPLETE': 3}
397 {'COMPLETE': 3}
398 {'COMPLETE': 3}
399 {'COMPLETE': 3}
400 {'COMPLETE': 3}
401 {'COMPLETE': 3}
402 {'COMPLETE': 3

In [ ]:
playback datalog mi.instrument.nortek.vector.ooicore.driver CE02SHBP-LJ01D-07-VEL3DC108 
qpid://guest/guest@ooiufs03?queue=Ingest.instrument_events 
    qpid://guest/guest@ooiufs03?queue=Ingest.instrument_particles 
        /rsn_cabled/DVT_Data/lj01d/VEL3DC108*20{15{102[012345678],12{2[89],3}},16010[1234]}* 2>&1 >
        ~/rsn_logs/CE02SHBP-LJ01D-07-VEL3DC108-4.log

qpid://guest/guest@ooiufs11?queue=Ingest.instrument_events qpid://guest/guest@ooiufs11?queue=Ingest.instrument_particles

In [21]:
### PURGE DATA - DON'T USE ON PRODUCTION
###
### WARNING
###
### WARNING - check the context BEFORE running this!!!
###
for refdes in ['CE04OSPS-PC01B-4D-PCO2WA105']:
    # TODO - add check for production - don't do it!
    print('Will purge ALL data from %s on %s. Uncomment the following line if you are sure and rerun.' % (refdes, m2m.context()))
    #response = m2m.purge(refdes)
    #print(response.json())

Will purge ALL data from CE04OSPS-PC01B-4D-PCO2WA105 on https://ooinet-dev-01.oceanobservatories.org danmergens@gmail.com. Uncomment the following line if you are sure and rerun.


In [43]:
# VEL3DC108 recovery
filemasks = [
            '/rsn_cabled/rsn_data/DVT_Data/lj01d/VEL3DC108_*.dat',
            '/rsn_cabled/rsn_data/DVT_Data/lj01d/VEL3DC108/*/*/VEL3DC*.dat',
            ]
vel3d_data_range = ('2014-09-24T00:00:00Z', '2019-03-25 05:24:02.094Z')
request = cabledRequestFactory(user, 'CE02SHBP-LJ01D-07-VEL3DC108', filemasks, data_range=vel3d_data_range)
#print json.dumps(request, indent=4)
response = m2m.playback(request)
print(response.reason, response.json())

TypeError: 'str' object is not callable

In [43]:
# VEL3DC107 recovery - gap #1
filemasks = [
            '/rsn_cabled/rsn_data/DVT_Data/lj01d/VEL3DC108/2018/09/VEL3DC*_201809*.dat',
            ]
vel3d_data_range = ('2018-09-05T10:25:04.262Z', '2018-09-14T20:15:55.305Z')
request = cabledRequestFactory(user, 'CE02SHBP-LJ01D-07-VEL3DC108', filemasks, data_range=vel3d_data_range)
#print(json.dumps(request, indent=4))
response = m2m.playback(request)
print(response.reason, response.json())

TypeError: 'str' object is not callable

In [43]:
# VEL3DC107 recovery - gap #2
filemasks = [
            '/rsn_cabled/rsn_data/DVT_Data/lj01d/VEL3DC108/2018/09/VEL3DC*.dat',
            '/rsn_cabled/rsn_data/DVT_Data/lj01d/VEL3DC108/2018/10/VEL3DC*.dat',
            ]
vel3d_data_range = ('2018-09-15T00:00:00Z', '2018-10-11 21:27:42.648Z')
request = cabledRequestFactory(user, 'CE02SHBP-LJ01D-07-VEL3DC108', filemasks, data_range=vel3d_data_range)
#print(json.dumps(request, indent=4))
response = m2m.playback(request)
print(response.reason, response.json())

TypeError: 'str' object is not callable